# Homework 3 - Which book would you recommend?

*Stefano D'Arrigo 1960500, Alessio Sentinelli, Iyuele Alemu Korsaye*

---

![goodreads image](./images/goodrreads.jpg)

## Notes before starting

In order to keep this notebook tidy and agile to read, the majority of the code we wrote to complete the tasks is not included here and is provided into the folder `scripts`. Nevertheless, the crucial pieces of code are directly executed or shown and commented inside this notebook. For further understanding of each operation and choice we made, please refer to the comments to the code.

---

## 1. Data collection

#### Loading libraries

In [2]:
from bs4 import BeautifulSoup as bs # library for pulling data out of HTML files
import requests # library used to  make HTTP requests simpler
import re # regular expression matching operations
from selenium import webdriver # to automate website testing.
import chromedriver_binary # for google chrome
import spacy # library for advanced natural language processing
from spacy_fastlang import LanguageDetector # to detect languages

### 1.1. Get the list of books

First, we open a txt file and extract the URL of the 30k books, using a for loop over the 300 pages and leveraging the request library we get access to the pages where the books are present.

Then using a third-party library, beautiful soup we can pull the data out of the HTML files and use the ‘lxml’ parser to extract the class "js tooltipTrigger tooltipTrigger" within a div tag, where the url of the books is present.
Finally we write within a loop function the collected 30k url on the text file we opened previously and close the file.



In [ ]:
f = open("url_list.txt","w") # open the new txt file
for k in range (1,301):  #301 for the pages
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=" + str(k)) # select the page
    soup = BeautifulSoup(page.content, features="lxml") # take the content from the page
    URL_con3 = soup.find_all('div', class_="js-tooltipTrigger tooltipTrigger") # to filter what we need 
    for j in range (0,100): #100 for the books in every page
        URL_str = str(URL_con3[j]) 
        list_split = URL_str.split(" ")
        result = list_split[5] 
        result_clean = result.split("\"")[1] # now we have the URL 
        f.write("https://www.goodreads.com" + result_clean + "\n") # write the URL on the txt file
f.close()

We obtained a txt file with 30k lines with each line containing the URL book, and the txt file is finally saved in the data folder.

### 1.2. Crawl books

The goal of this task is to retrieve all the `HTML` pages of the books, reading the `url_list.txt` file that we created in the previous task.

In order to bypass eventual security measures against scraping, we leveraged the library `selenium`, which provides a full and automatized web client agent. 

To complete this task, we wrote a class `DataCollector`, included in `data_collection.py`. The methods of this class receive the user's input, compute the offset from which start reading the URLs file and save the HTML pages. 

The core business of this class is included into the following method:

~~~python
def __save_html_pages(self, start_from, stop_at):
        """
        Start collecting from line start_from and stop at line stop_at.
        """
        with open(os.path.join(self.root_dir, 'url_list.txt'), 'r') as urls_file:
            try:
                urls = urls_file.readlines()[start_from : ] # select the line from which start collecting
            except:
                print('Error: reached file end!')
                exit(-1)
            for url, i in zip(urls, tqdm(range(start_from, stop_at))): # 
                if i % 100 == 0:
                    self.__make_dir(i // 100 + 1)
                try:
                    driver.get(url)
                    page_html = driver.page_source
                    with open(os.path.join(self.html_dir, f'article_{i + 1:05d}.html'), 'w') as out_file:
                        out_file.write(page_html)
                except:
                    with open('./log/log.csv', 'a') as log:
                        log.write(f'[{datetime.datetime.now()}], {i+1}, {url}\n')
                    continue
            driver.close()
~~~

Using the parameter `start_from`, the user can decide from which document start crawling. The eventual errors in retrieving the pages were annotated in a log file and handled manually after the execution of the script.

The output of this method are the collected data, structured in the following way:

```
- html/
    - 1/
        - article_00001.html
        - article_00002.html
        - ...
        - article_00100.html
    - 2/
        - article_00101.html
        - ...
        - article_00200.html
    - ...
    - 300/
        - article_29901.html
        - ...
        - article_30000.html
```

### 1.3 Parse downloaded pages

Once we have accessed the HTML content of all the 30 000 books, we are left with the task of parsing the data. Since most of the HTML data is nested, we cannot extract data simply through string processing. One needs a parser which can create a nested/tree structure of the HTML data. There are many HTML parser libraries available, the one we have used in our function (book_scraping) is ‘lxml’ parser.

Now, we need to navigate and search the parse tree that we created and for this task, we will be using another third-party python library, Beautiful Soup. It is a Python library for pulling data out of HTML and XML files. A really nice feature  about the BeautifulSoup library is that it is built on the top of the HTML parsing libraries like, lxml, html5lib parser, etc. so  that BeautifulSoup object and the parser library can be created at the same time.
soup = BeautifulSoup(html_source, features='lxml')

Now, we are ready to extract all the relevant data from the HTML content that are crucial for building a book recommendation engine. The soup object contains all the data in the nested structure which can be programmatically extracted using the ‘book scraping’ function, that we have created to retrieve and save for each 30k book all the relevant information (book title, book series, book author, rating value, rating count, review count, plot, number of pages, published date, characters, settings and URL)


In [1]:
def book_scraping(html_source): # this takes the html content and returns a list with the useful info
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe(LanguageDetector())

    soup = BeautifulSoup(html_source, features='lxml') # instantiate a BeautifulSoup object for HTML parsing

    bookTitle = soup.find_all('h1', id='bookTitle')[0].contents[0].strip() # get the book title

    # if bookSeries is not present, then set it to the empty string
    try:
        bookSeries = soup.find_all('h2', id='bookSeries')[0].contents[1].contents[0].strip()[1:-1]
    except:
        bookSeries = ''

    # if bookAuthors is not present, then set it to the empty string
    try:
        bookAuthors = soup.find_all('span', itemprop='name')[0].contents[0].strip()
    except:
        bookAuthors = ''
    
    # the plot of the book is essential; if something goes wrong with the plot, raise an error
    try:
        Plot = soup.find_all('div', id='description')[0].contents  # get the main tag where the plot is found 
        filter_plot = list(filter(lambda i: i!='\n', Plot))  # filter the plot by removing tags that doesn’t contain the description 
        if len(filter_plot) == 1:    
            Plot = filter_plot[0].text
        else:                          # getting all the plot within the tag
            Plot = filter_plot[1].text  
        doc = nlp(Plot)
        if doc._.language != 'en':          
            raise Exception # if the plot is not in english, raise an error
    except:
        raise # pass the error to the caller function


    # if NumberofPages is not present, then set it to the empty string
    try:
        NumberofPages = soup.find_all('span', itemprop='numberOfPages')[0].contents[0].split()[0]
    except:
        NumberofPages = ''
    
    # if ratingValue is not present, then set it to the empty string
    try:
        ratingValue = soup.find_all('span', itemprop='ratingValue')[0].contents[0].strip()
    except:
        ratingValue = ''
    
    # if rating_reviews is not present, then set it to the empty string
    try:
        ratings_reviews = soup.find_all('a', href='#other_reviews')
        for i in ratings_reviews:
            if i.find_all('meta',itemprop='ratingCount'):
                ratingCount = i.contents[2].split()[0]
            if i.find_all('meta',itemprop='reviewCount'):
                reviewCount = i.contents[2].split()[0]
    except:
        ratings_reviews = ''

    # if Published is not present, then set it to the empty string
    try:        
        pub = soup.find_all('div', class_='row')[1].contents[0].split()[1:4]
        Published = ' '.join(pub) # join the list of publishers
    except:
        Published = ''
    
    # if Character is not present, then set it to the empty string
    try:
        char = soup.find_all('a', href=re.compile('characters')) # find the regular expression(re) 'characters' within the attribute href 
        if len(char) == 0:
            Characters = '' # no characters in char
        else:
            Characters = ', '.join([i.contents[0] for i in char])
    except:
        Characters = '' # something went wrong with char
    
    # if Setting is not present, then set it to the empty string
    try:
        sett = soup.find_all('a', href=re.compile('places')) # find the regular expression(re) 'places' within the attribute href 
        if len(sett) == 0:
            Setting = ''
        else:
            Setting = ', '.join([i.contents[0] for i in sett])
    except:
        Setting = '' # something went wrong with Setting
    
    # get the URL to the page
    Url = soup.find_all('link', rel='canonical')[0].get('href')

    return [bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url]


The output of the function is structured in a manner that for each book, the extracted relevant information are in a tab separated values extensions ready to be feed-in for the next stage.
During the scraping procedure, if the information we were seeking was not available then an empty string is returned and also books with a description written in a different language than in English were discarded.


## 2. Search Engine

Once we have collected all the raw HTML pages and scraped the target information, we apply some natural language processing (NLP) techniques in order to create the files which the search engine will work on.

The text tools we use are available in the `ntlk` library:

In [1]:
import nltk # Natural Language Toolkit
nltk.download('punkt') # a library for tokenizing texts
nltk.download('stopwords') # a library containg a list of stop-words

First thing first, it is worth to tokenize the text, i.e. split it into a list of single words, according to punctuation and words formation rules:

In [2]:
def tokenize(text:str):
    return nltk.word_tokenize(text)

Then, we filter the words that contain alphanumeric characters only and we convert all of them into lower case: 

In [3]:
def alphanum(text:list): 
    text_result = []
    for w in text:
        if w.isalnum():
            text_result.append(w.lower()) # to transform all the characters in lower case
    return text_result

The input text is full of very common words which, then, are very poor of information (according to the concept of self-information in information theory). These words are known as "stop words". The following function aims to remove all of them, comparing each word with a pre-build stopwords list: 

In [4]:
def stopwords(text:list):
    text_result = []
    stop_words = nltk.corpus.stopwords.words('english') # we select the stopwords of english language
    for w in text:
        if w not in stop_words:
            text_result.append(w) # we filter out the stopwords
    return text_result

The last step of the natural language processing pipeline that we reckon is useful at this point is the mapping of each word to its corresponding root, so that words derivated from the same root are mapped into it; e.g. verbs forms are mapped into the base form of the verb. This task can be carried out by two different techniques: the stemming and the lemmatization. The former applies to each word some fixed rules of words formation to remove prefixes and suffixes; it sometimes lacks in accuracy, giving in output not meaningful words, but it is quite fast. On the contrary, the latter is way more accurate because it considers a language’s vocabulary to apply a morphological analysis to words, but its application is slower. For the purpose of this excercise, a stemmer is enought. Again, it is easy to plug-in a lemmatizer instead of the stemmer for future needs, thanks to the modular nature of our code.

In [5]:
def stemming(text:list):
    stemmer = nltk.stem.PorterStemmer()
    return [stemmer.stem(w) for w in text]

Finally, to summarize and to clarify, the NLP pipeline we follow is:
1. tokenization
2. alphanumeric filtering and lower case conversion
3. stopwords removing
4. stemming

The function below summarizes this pipeline, taking as input a row text and giving as output the processed text:

In [7]:
def pre_process(text:str):
    return ' '.join(stemming(stopwords(alphanum(tokenize(text)))))

For the sake of grouping the functions above, we put them into the class `scripts.utilities.TextTools`.

### 2.1. Conjunctive query

In this part, we import the following classes. The class `IndexBuilder` holds the logic to concatenate the books' dataset, to create the vocabulary and the inverted indexes.

The class `TextTools`, as said before, is a utility class for applying the NLP.

In [1]:
from scripts.index_creation import IndexBuilder # to create inverted indexes
from scripts.utilities import TextTools # for text processing
import pandas as pd

pd.set_option("display.max_colwidth", 102)

#### 2.1.1) Create your index!

We start getting an instance of the class `IndexBuilder`.

In [2]:
index_builder = IndexBuilder()

With the following code line we concatenate into a dataset the content of all the .tsv files.

In [ ]:
dataset = index_builder.concatenate_dataset('./data/tsv/*/*.tsv', fields=None) 

In order to avoid errors, we filter out the records that might have no plot. Even thought we filtered the books without a valid description, two books appear to have a blank description, perhaps due to the structure of `pandas.DataFrame`.

In [ ]:
dataset = dataset[dataset['Plot'].notnull()]

In [6]:
dataset.head()

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberofPages,Published,Characters,Setting,Url,file_num
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,"6,413,302","172,615","Could you survive on your own in the wild, with every one out to make sure you don't live to see t...",374.0,September 14th 2008,"Katniss Everdeen, Peeta Mellark, Cato (Hunger Games), Primrose Everdeen, Gale Hawthorne, Effie Tri...","District 12, Panem, Capitol, Panem, Panem",https://www.goodreads.com/book/show/2767052-the-hunger-games,1
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,J.K. Rowling,4.50,"2,527,001","42,768",There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why els...,870.0,September 2004 by,"Sirius Black, Draco Malfoy, Ron Weasley, Petunia Dursley, Vernon Dursley, Dudley Dursley, Severus ...","Hogwarts School of Witchcraft and Wizardry, London, England",https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix,2
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,"4,530,963","91,866",The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that...,324.0,May 23rd 2006,"Scout Finch, Atticus Finch, Jem Finch, Arthur Radley, Mayella Ewell, Aunt Alexandra, Bob Ewell, Ca...","Maycomb, Alabama",https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird,3
3,Pride and Prejudice,NaN,Jane Austen,4.26,"3,020,392","67,869","Alternate cover edition of ISBN 9780679783268Since its immediate success in 1813, Pride and Prejud...",279.0,October 10th 2000,"Mr. Bennet, Mrs. Bennet, Jane Bennet, Elizabeth Bennet, Mary Bennet, Kitty Bennet, Lydia Bennet, L...","United Kingdom, Derbyshire, England, England, Hertfordshire, England",https://www.goodreads.com/book/show/1885.Pride_and_Prejudice,4
4,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,"4,993,492","104,954","About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part ...",501.0,September 6th 2006,"Edward Cullen, Jacob Black, Laurent, Renee, Bella Swan, Billy Black, Esme Cullen, Alice Cullen, Ja...","Forks, Washington, Phoenix, Arizona, Washington (state)",https://www.goodreads.com/book/show/41865.Twilight,5


<br>

The column `file_num` contains the position in the books' ranking. The `dataset` is sorted by this column.

Then, we make a copy of the dataset not to modify the column `Plot`, that we will use later. On the copy we apply the NLP techniques.

In [ ]:
dataset_preprocessed = dataset.copy()
dataset_preprocessed['Plot'] = dataset_preprocessed['Plot'].map(TextTools.pre_process)

At this point, we create the vocabulary file using `IndexBuilder.create_vocabulary(..)` and we save it into the folder `./data`.

In [7]:
index_builder.create_vocabulary(dataset_preprocessed)

Then, we create the first inverted index in order to create our search engine.

In [8]:
index_builder.create_index(dataset_preprocessed)

The inverted index has the format

```json
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```
and is saved as a JSON file. We chose this file extension to ease the reading/writing operations, since the conversion between JSON and Python `dict()` is straightforward.

#### 2.1.2) Execute the query

We wrote the class `SearchEngine` to handle the research of the queries. 

In [6]:
from scripts.search_engine import SearchEngine

search_engine = SearchEngine()
search_engine.select_mode(use_tfidf=False)

With the method `select_mode(..)` we change the behaviour of the object `search_engine`: for the present task we set the `use_tfidf` parameter to `False` to leverage the simple inverted index we created in the previous subpoint.
<br><br><br>
At this point, we can test the search engine:

In [7]:
query = input()
result = search_engine.search(query, dataset)

hunger games


In [8]:
result

,bookTitle,Plot,Url
0,The Hunger Games,"Could you survive on your own in the wild, with every one out to make sure you don't live to see t...",https://www.goodreads.com/book/show/2767052-the-hunger-games
183,The Hunger Games Trilogy Boxset,"The extraordinary, ground breaking New York Times bestsellers The Hunger Games and Catching Fire, ...",https://www.goodreads.com/book/show/7938275-the-hunger-games-trilogy-boxset
219,Catching Fire,"SPARKS ARE IGNITING.FLAMES ARE SPREADING.AND THE CAPITAL WANTS REVENGE.Against all odds, Katniss E...",https://www.goodreads.com/book/show/6148028-catching-fire
315,Mockingjay,"The final book in the ground-breaking HUNGER GAMES trilogy, this new foiled edition of MOCKINGJAY ...",https://www.goodreads.com/book/show/7260188-mockingjay
1301,Crossing the Seas: A Diary of My Thoughts,"About the book (A teaser) ""Crossing the Seas: A Diary of My Thoughts"" ""We can forgive a child who ...",https://www.goodreads.com/book/show/16243767-crossing-the-seas
1421,Sliding on the Snow Stone,It is astonishing that anyone lived this story. It is even more astonishing that anyone survived i...,https://www.goodreads.com/book/show/12853168-sliding-on-the-snow-stone
2261,Best Served Cold,Springtime in Styria. And that means war. Springtime in Styria. And that means revenge.There have ...,https://www.goodreads.com/book/show/2315892.Best_Served_Cold
2868,Morning Star,#1 NEW YORK TIMES BESTSELLER • Red Rising thrilled readers and announced the presence of a talente...,https://www.goodreads.com/book/show/18966806-morning-star
4457,The Hunger Games: Official Illustrated Movie Companion,Go behind the scenes of the making of The Hunger Games with exclusive images and interviews. From ...,https://www.goodreads.com/book/show/11742691-the-hunger-games
4508,"The Maze Runner Trilogy (Maze Runner, #1-3)","The perfect gift for fans of The Hunger Games and Divergent, this boxed set of the paperback editi...",https://www.goodreads.com/book/show/17292676-the-maze-runner-trilogy


We chose to show the first 10 results in the same order they appear in the site's list. As we can see, the search gives us the books that contain both "hunger" and "games" in the plot.

### 2.2) Conjunctive query & Ranking score

Now, for each document and word we take in consideration the TF-IDF score and we select the top k books that have the highest cosine similarity.



#### 2.2.1) Inverted index

We create the second inverted index.

In [13]:
index_builder.create_index_tfidf(dataset_preprocessed)

25821it [00:24, 1064.29it/s]


The inverted index has the format

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}
```
and, as before, is saved into JSON file.

#### 2.2.2) Execute the query

For this task, we switch the mode of the `search_engine`, so that it works on the second inverted index.

In [9]:
search_engine.select_mode(use_tfidf=True)

<br><br>
Now, we are ready to test our search engine again. 

In [10]:
query = input()
result = search_engine.search(query, dataset, k=10)
result

hunger games


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

<br><br>
The 

## 3. Define a new score!

$
newScore = similarity_{Plot} \cdot 0.6 + score_{Title} \cdot 0.1 + score_{Authors} \cdot 0.07 + score_{Series}
$

In [18]:
query = input()
additional_query = input()
search_engine.search(query, dataset, k=10, new_score=True, additional_query=additional_query)

hunger games
mockingjay


,bookTitle,Plot,Url,Similarity
0,The Hunger Games,"Could you survive on your own in the wild, with every one out to make sure you don't live to see t...",https://www.goodreads.com/book/show/2767052-the-hunger-games,0.581
183,The Hunger Games Trilogy Boxset,"The extraordinary, ground breaking New York Times bestsellers The Hunger Games and Catching Fire, ...",https://www.goodreads.com/book/show/7938275-the-hunger-games-trilogy-boxset,0.398
315,Mockingjay,"The final book in the ground-breaking HUNGER GAMES trilogy, this new foiled edition of MOCKINGJAY ...",https://www.goodreads.com/book/show/7260188-mockingjay,0.384
4457,The Hunger Games: Official Illustrated Movie Companion,Go behind the scenes of the making of The Hunger Games with exclusive images and interviews. From ...,https://www.goodreads.com/book/show/11742691-the-hunger-games,0.265
4949,"SAMPLER ONLY: Catching Fire (The Hunger Games, #2)","Against all odds, Katniss Everdeen has won the annual Hunger Games with fellow district tribute Pe...",https://www.goodreads.com/book/show/20349441-sampler-only,0.263
6437,Catching Fire: The Official Illustrated Movie Companion,"Catching Fire, the New York Times bestseller by Suzanne Collins, is now a major motion picture -- ...",https://www.goodreads.com/book/show/17623910-catching-fire,0.252
9310,Eeny Meeny,"The ""dark, twisted, thought-provoking"" (#1 New York Times bestseller Tami Hoag) international best...",https://www.goodreads.com/book/show/23398806-eeny-meeny,0.192
11199,Throne of Glass Collection,"Perfect for the Fans of Hunger Games, Game Of Throne, Throne Of Glass Series Collection 3 Books Se...",https://www.goodreads.com/book/show/26309016-throne-of-glass-collection,0.187
23555,"The Dust Lands Trilogy: Blood Red Road; Rebel Heart; Raging Star (Dust Lands, #1-3)","All three books in the highly praised Dust Lands trilogy, which MTV’s Hollywood Crush blog called ...",https://www.goodreads.com/book/show/24885710-the-dust-lands-trilogy,0.117
25651,The Hunger Games Tribute Guide,The New York Times bestselling Hunger Games is now a major motion picture—and here is the ultimate...,https://www.goodreads.com/book/show/13027304-the-hunger-games-tribute-guide,0.104


## 5. Algorithmic Question

This problem can be solved with a recursive approach.

Given a string $S$ and $i\in [0,length(S)]$, let $X[i]$ be the length of the longest increasing subsequence ending at position $i$. 

Then, $X[i]=1+\max\{X[j]; j\in[0,i-1] : S[j]<S[i]\}$.

The recursive solution is the following:

In [7]:
max_len = 1

def max_length_recursive(s, i):
    global max_len
    max_len_i = 1
    for j in range(0, i):
        res = max_length_recursive(s, j)
        if s[j] < s[i]:
            if res+1 > max_len_i:
                max_len_i = res + 1
    max_len = max(max_len, max_len_i)
    return max_len_i

In [8]:
def max_length(s):
    max_len = 0
    for i in range(1, len(s)):
        res = max_length_recursive(s, i)
        if res > max_len:
            max_len = res
    return max_len

In [9]:
s = 'ZQABWARSTA' # 'CADFECEILGJHABNOPSTIRYOEABILCNR'
max_length_recursive(s, len(s)-1)
max_len

5

The time complexity for the recursive technique is of $O(2^n)$, which is exponential. For larger values of n, there will be many values for which the function has to be repeated and thus this approach results to be inefficient and may not give a result in reasonable time. For example, the algorithm gives the right answer with short strings like `ZQABWARSTA`, while it doesn't terminate with the string `CADFECEILGJHABNOPSTIRYOEABILCNR`. Indeed, in the latter case, the time taken by the algorithm is $2^{31}$!

<center>
$
T(n)=c + \sum_{i=1}^{n-1}T(i) = c + T(n-1) + \sum_{i=1}^{n-2}T(i) =\\
= c + \sum_{i=1}^{n-2}T(i) + \sum_{i=1}^{n-2}T(i) =\\
= c + 2 \cdot \left(\sum_{i=1}^{n-2}T(i)\right) =\\
= c + 2 \cdot \left(T(n-2) + \sum_{i=1}^{n-3}T(i)\right) =\\
= c + 2 \cdot \left(c + \sum_{i=1}^{n-3}T(i) + \sum_{i=1}^{n-3}T(i)\right) =\\
= c + 4 \cdot \left(\sum_{i=1}^{n-3}T(i)\right) = ... \approx 2^n
$
</center>


On the contrary, using  the bottom-up approach of the dynamic programming we can improve the time complexity of the algorithm to the order of $O(n^2)$. This approch takes into account the result of the subproblems, without recomputing them each time. Then, the complexity is only due to the 2 nested loops.